# DeepQL with tensorflow

https://github.com/AdrianUng/keras-triplet-loss-mnist/blob/master/Triplet_loss_KERAS_semi_hard_from_TF.ipynb

In [1]:
# from __future__ import print_function, division
import keras
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

from keras_bert import load_vocabulary
import random

from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import tensorflow as tf

from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam
import _pickle as pickle
    
# %matplotlib inline

Using TensorFlow backend.


## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = int(os.environ['epochs'])
freeze_train = .1
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = os.environ['base']
METHOD = 'deepTL_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
token_dict = load_vocabulary(vocab_path)

In [10]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [11]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

#### Loading bug ids in memory

In [12]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


98070

#### Dicionário de títulos e descrições

In [13]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 10.2 s, sys: 1.09 s, total: 11.3 s
Wall time: 11.3 s


#### Hashing bugs by buckets

In [14]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [15]:
# path_train='train_chronological', path_test='test_chronological'
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

In [16]:
baseline.train_data[:10]

[[6036, 35039],
 [26311, 15091],
 [5142, 34523],
 [8323, 9594],
 [76676, 88775],
 [59725, 71546],
 [121756, 116964],
 [32936, 30911],
 [15257, 16297],
 [99719, 106395]]

#### Recovery bug ids from train

In [17]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [18]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

# Generating tiple of batches

In [19]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '1\n',
 'component': '48\n',
 'creation_ts': '2003-09-19 16:09:00 +0000',
 'delta_ts': '2013-08-07 15:00:39 +0000',
 'description': '[CLS] try the next experiment , in hebrew , on any o ##oo version of the recent ones : 1 ) start a paragraph in bold . 2 ) put a dash " - " after it . bold or not , doesn \' t matter . 3 ) change back to regular weight and write a nice paragraph . 4 ) save as a . doc document ( rt ##f may also have a problem ) . 5 ) open your . doc file in ms ##word and find out that the text you had in bold is written backwards , if you \' re lucky . this doesn \' t effect the document in o ##oo , but still , it pose a problem for compatibility . [SEP]',
 'description_segment': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


### Generating the batch test

In [20]:
"Train ", len(baseline.dup_sets_train)

('Train ', 14508)

In [21]:
# data - path
# batch_size - 128
# n_neg - 1
def batch_iterator(self, retrieval, model, data, dup_sets, bug_ids, 
                   batch_size, n_neg, issues_by_buckets, TRIPLET_HARD=False, FLOATING_PADDING=False):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_features = {'title' : [], 'desc' : [], 'info' : []}
    n_train = len(data)

    batch_triplets, batch_bugs_anchor, batch_bugs_pos, batch_bugs_neg, batch_bugs = [], [], [], [], []

    all_bugs = list(issues_by_buckets.keys())
    buckets = retrieval.buckets

    for offset in range(batch_size):
        anchor, pos = data[offset][0], data[offset][1]
        batch_bugs_anchor.append(anchor)
        batch_bugs_pos.append(pos)
        batch_bugs.append(anchor)
        batch_bugs.append(pos)
        #batch_bugs += dup_sets[anchor]

    for anchor, pos in zip(batch_bugs_anchor, batch_bugs_pos):
        while True:
            neg = self.get_neg_bug(anchor, buckets[issues_by_buckets[anchor]], issues_by_buckets, all_bugs)
            bug_anchor = self.bug_set[anchor]
            bug_pos = self.bug_set[pos]
            if neg not in self.bug_set:
                continue
            batch_bugs.append(neg)
            batch_bugs_neg.append(neg)
            bug_neg = self.bug_set[neg]
            break
        
        # triplet bug and master
        batch_triplets.append([anchor, pos, neg])
    
    random.shuffle(batch_bugs)
    title_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_T), 0)
    description_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_D), 0)
    for i, bug_id in enumerate(batch_bugs):
        bug = self.bug_set[bug_id]
        self.read_batch_bugs(batch_features, bug, index=i, title_ids=title_ids, description_ids=description_ids)

    batch_features['title'] = { 'token' : np.array(batch_features['title']), 'segment' : title_ids }
    batch_features['desc'] = { 'token' : np.array(batch_features['desc']), 'segment' : description_ids }
    batch_features['info'] = np.array(batch_features['info'])
    
    sim = np.asarray([issues_by_buckets[bug_id] for bug_id in batch_bugs])

    input_sample = {}

    input_sample = { 'title' : batch_features['title'], 
                        'description' : batch_features['desc'], 
                            'info' : batch_features['info'] }

    return batch_triplets, input_sample, sim #sim

In [22]:
# %%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_sim = batch_iterator(baseline, retrieval, None, 
                                                                                      baseline.train_data, 
                                                                                      baseline.dup_sets_train,
                                                                                      bug_train_ids,
                                                                                      batch_size_test, 1,
                                                                                      issues_by_buckets)

validation_sample = [valid_input_sample['title']['token'], valid_input_sample['title']['segment'], 
                   valid_input_sample['description']['token'], valid_input_sample['description']['segment'],
                   valid_input_sample['info'], valid_sim]

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

In [23]:
valid_input_sample['title']['token'].shape, \
valid_input_sample['description']['token'].shape, \
valid_input_sample['title']['segment'].shape, \
valid_input_sample['description']['segment'].shape, \
valid_input_sample['info'].shape, valid_sim.shape

((384, 20), (384, 20), (384, 20), (384, 20), (384, 738), (384,))

### Validar entrada

In [24]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [25]:
"Test ", len(baseline.test_data)

('Test ', 8265)

## Pre-trained embeddings

Loading pretrained word vectors

### Glove

In [26]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [27]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 20031'

## Experiment

## Towards Accurate Duplicate Bug Retrieval Using Deep Learning Techniques

https://github.com/tqtg/DuplicateBugFinder

### BERT

In [31]:
def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    layers = ['Encoder-{}-MultiHeadSelfAttention-Adapter', 'Encoder-{}-FeedForward-Adapter', 
     'Encoder-{}-MultiHeadSelfAttention-Norm', 'Encoder-{}-FeedForward-Norm']
    outputs = []
    for i in range(1, 13):
        outputs += [ model.get_layer(layer.format(layer_num)).output for layer in layers ]
    outputs = Average()(outputs)
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
    outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [32]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [33]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [34]:
def quintet_loss(y_true, y_pred):
    return triplet_loss(y_pred)

def triplet_loss(vects):
    #del y_true
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = vects[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

In [35]:
def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [36]:
def max_margin_objective(encoded_anchor, decay_lr=1):
    
    input_labels = Input(shape=(1,), name='input_label')    # input layer for labels
    inputs = np.concatenate([encoded_anchor.input, [input_labels]], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    
    output = concatenate([input_labels, encoded_anchor])  # concatenating the labels + embeddings
    
    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    # optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    
    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=quintet_loss) 
    # metrics=[pos_distance, neg_distance, custom_margin_loss]

    return similarity_model

In [37]:
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [38]:
# Domain to use
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
METHOD = 'deepTL_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [39]:
# %%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')

similarity_model = max_margin_objective(encoded_anchor, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, encoded_anchor, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append([h])
    result['test'].append([h_validation])
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, recall@25: {:.2f}".format(epoch+1, h, h_validation, recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}".format(epoch+1, h, h_validation))
    loss = h
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

#experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
#experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________

=> result saved!
Epoch: 30 Loss: 0.08, Loss_test: 0.05
Epoch: 31 Loss: 0.03, Loss_test: 0.05
Epoch: 32 Loss: 0.08, Loss_test: 0.05
Epoch: 33 Loss: 0.13, Loss_test: 0.05
Epoch: 34 Loss: 0.04, Loss_test: 0.05
Epoch: 35 Loss: 0.08, Loss_test: 0.05
Epoch: 36 Loss: 0.13, Loss_test: 0.05
Epoch: 37 Loss: 0.04, Loss_test: 0.05
Epoch: 38 Loss: 0.07, Loss_test: 0.05
Epoch: 39 Loss: 0.04, Loss_test: 0.05
=> result saved!
Epoch: 40 Loss: 0.07, Loss_test: 0.05
Epoch: 41 Loss: 0.12, Loss_test: 0.05
Epoch: 42 Loss: 0.05, Loss_test: 0.04
Epoch: 43 Loss: 0.04, Loss_test: 0.04
Epoch: 44 Loss: 0.02, Loss_test: 0.04
Epoch: 45 Loss: 0.04, Loss_test: 0.04
Epoch: 46 Loss: 0.04, Loss_test: 0.04
Epoch: 47 Loss: 0.06, Loss_test: 0.04
Epoch: 48 Loss: 0.12, Loss_test: 0.04
Epoch: 49 Loss: 0.03, Loss_test: 0.04
=> result saved!
Epoch: 50 Loss: 0.10, Loss_test: 0.04
Epoch: 51 Loss: 0.05, Loss_test: 0.04
Epoch: 52 Loss: 0.14, Loss_test: 0.04
Epoch: 53 Loss: 0.07, Loss_test: 0.04
Epoch: 54 Loss: 0.03, Loss_test: 0.04

In [40]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepTL_100.txt'

In [41]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [42]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

Saved model 'modelos/model_bert_preprocessing_deepTL_100_feature_100epochs_64batch(openoffice).h5' to disk


In [43]:
len(result['train']), len(result['test'])

(100, 100)

In [44]:
model = similarity_model.get_layer('concatenate_1')
output = model.output
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss=quintet_loss)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [45]:
# Domain to use
METHOD = 'deepTL_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [46]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, model, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    

    h = model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append([h])
    result['test'].append([h_validation])
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == end_train )):
        save_loss(result)
    
    print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}".format(epoch+1, h, h_validation))

Epoch: 101 Loss: 0.04, Loss_test: 0.03
Epoch: 102 Loss: 0.02, Loss_test: 0.03
Epoch: 103 Loss: 0.07, Loss_test: 0.03
Epoch: 104 Loss: 0.04, Loss_test: 0.03
Epoch: 105 Loss: 0.04, Loss_test: 0.03
Epoch: 106 Loss: 0.05, Loss_test: 0.04
Epoch: 107 Loss: 0.18, Loss_test: 0.04
Epoch: 108 Loss: 0.04, Loss_test: 0.04
Epoch: 109 Loss: 0.04, Loss_test: 0.04
=> result saved!
Epoch: 110 Loss: 0.14, Loss_test: 0.04
Epoch: 111 Loss: 0.06, Loss_test: 0.04
Epoch: 112 Loss: 0.06, Loss_test: 0.04
Epoch: 113 Loss: 0.09, Loss_test: 0.04
Epoch: 114 Loss: 0.02, Loss_test: 0.04
Epoch: 115 Loss: 0.09, Loss_test: 0.04
Epoch: 116 Loss: 0.03, Loss_test: 0.04
Epoch: 117 Loss: 0.03, Loss_test: 0.04
Epoch: 118 Loss: 0.08, Loss_test: 0.04
Epoch: 119 Loss: 0.02, Loss_test: 0.04
=> result saved!
Epoch: 120 Loss: 0.07, Loss_test: 0.04
Epoch: 121 Loss: 0.04, Loss_test: 0.04
Epoch: 122 Loss: 0.02, Loss_test: 0.04
Epoch: 123 Loss: 0.09, Loss_test: 0.03
Epoch: 124 Loss: 0.06, Loss_test: 0.03
Epoch: 125 Loss: 0.06, Loss_te

Epoch: 303 Loss: 0.03, Loss_test: 0.01
Epoch: 304 Loss: 0.03, Loss_test: 0.01
Epoch: 305 Loss: 0.03, Loss_test: 0.01
Epoch: 306 Loss: 0.01, Loss_test: 0.01
Epoch: 307 Loss: 0.01, Loss_test: 0.01
Epoch: 308 Loss: 0.01, Loss_test: 0.01
Epoch: 309 Loss: 0.02, Loss_test: 0.01
=> result saved!
Epoch: 310 Loss: 0.02, Loss_test: 0.01
Epoch: 311 Loss: 0.01, Loss_test: 0.01
Epoch: 312 Loss: 0.01, Loss_test: 0.01
Epoch: 313 Loss: 0.03, Loss_test: 0.01
Epoch: 314 Loss: 0.01, Loss_test: 0.01
Epoch: 315 Loss: 0.01, Loss_test: 0.01
Epoch: 316 Loss: 0.01, Loss_test: 0.01
Epoch: 317 Loss: 0.01, Loss_test: 0.01
Epoch: 318 Loss: 0.01, Loss_test: 0.01
Epoch: 319 Loss: 0.02, Loss_test: 0.01
=> result saved!
Epoch: 320 Loss: 0.01, Loss_test: 0.01
Epoch: 321 Loss: 0.02, Loss_test: 0.01
Epoch: 322 Loss: 0.01, Loss_test: 0.01
Epoch: 323 Loss: 0.02, Loss_test: 0.01
Epoch: 324 Loss: 0.01, Loss_test: 0.01
Epoch: 325 Loss: 0.01, Loss_test: 0.01
Epoch: 326 Loss: 0.03, Loss_test: 0.01
Epoch: 327 Loss: 0.01, Loss_te

Epoch: 505 Loss: 0.02, Loss_test: 0.00
Epoch: 506 Loss: 0.01, Loss_test: 0.00
Epoch: 507 Loss: 0.01, Loss_test: 0.00
Epoch: 508 Loss: 0.02, Loss_test: 0.00
Epoch: 509 Loss: 0.01, Loss_test: 0.00
=> result saved!
Epoch: 510 Loss: 0.01, Loss_test: 0.00
Epoch: 511 Loss: 0.01, Loss_test: 0.00
Epoch: 512 Loss: 0.00, Loss_test: 0.00
Epoch: 513 Loss: 0.01, Loss_test: 0.00
Epoch: 514 Loss: 0.02, Loss_test: 0.00
Epoch: 515 Loss: 0.01, Loss_test: 0.00
Epoch: 516 Loss: 0.02, Loss_test: 0.00
Epoch: 517 Loss: 0.01, Loss_test: 0.00
Epoch: 518 Loss: 0.00, Loss_test: 0.00
Epoch: 519 Loss: 0.01, Loss_test: 0.00
=> result saved!
Epoch: 520 Loss: 0.00, Loss_test: 0.00
Epoch: 521 Loss: 0.01, Loss_test: 0.00
Epoch: 522 Loss: 0.01, Loss_test: 0.00
Epoch: 523 Loss: 0.01, Loss_test: 0.00
Epoch: 524 Loss: 0.01, Loss_test: 0.00
Epoch: 525 Loss: 0.02, Loss_test: 0.00
Epoch: 526 Loss: 0.00, Loss_test: 0.00
Epoch: 527 Loss: 0.01, Loss_test: 0.00
Epoch: 528 Loss: 0.02, Loss_test: 0.00
Epoch: 529 Loss: 0.00, Loss_te

Epoch: 707 Loss: 0.00, Loss_test: 0.00
Epoch: 708 Loss: 0.01, Loss_test: 0.00
Epoch: 709 Loss: 0.01, Loss_test: 0.00
=> result saved!
Epoch: 710 Loss: 0.00, Loss_test: 0.00
Epoch: 711 Loss: 0.01, Loss_test: 0.00
Epoch: 712 Loss: 0.00, Loss_test: 0.00
Epoch: 713 Loss: 0.01, Loss_test: 0.00
Epoch: 714 Loss: 0.04, Loss_test: 0.00
Epoch: 715 Loss: 0.01, Loss_test: 0.00
Epoch: 716 Loss: 0.01, Loss_test: 0.00
Epoch: 717 Loss: 0.01, Loss_test: 0.00
Epoch: 718 Loss: 0.01, Loss_test: 0.00
Epoch: 719 Loss: 0.01, Loss_test: 0.00
=> result saved!
Epoch: 720 Loss: 0.01, Loss_test: 0.00
Epoch: 721 Loss: 0.01, Loss_test: 0.00
Epoch: 722 Loss: 0.02, Loss_test: 0.00
Epoch: 723 Loss: 0.00, Loss_test: 0.00
Epoch: 724 Loss: 0.01, Loss_test: 0.00
Epoch: 725 Loss: 0.00, Loss_test: 0.00
Epoch: 726 Loss: 0.00, Loss_test: 0.00
Epoch: 727 Loss: 0.01, Loss_test: 0.00
Epoch: 728 Loss: 0.02, Loss_test: 0.00
Epoch: 729 Loss: 0.01, Loss_test: 0.00
=> result saved!
Epoch: 730 Loss: 0.00, Loss_test: 0.00
Epoch: 731 Lo

In [47]:
len(result['train']), len(result['test'])

(900, 900)

In [48]:
encoded = model.get_layer('merge_features_in')
output = encoded.output
inputs = similarity_model.inputs[:-1]
encoded_anchor = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [49]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepTL_1000_feature1000epochs_64batch(openoffice)'

In [50]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

Saved model 'modelos/model_bert_preprocessing_deepTL_1000_feature_1000epochs_64batch(openoffice).h5' to disk


'Model saved'

In [51]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 1, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, recall@25: {:.2f}".format(epoch+1, h, h_validation, recall))


Epoch: 900 Loss: 0.01, Loss_test: 0.00, recall@25: 0.66


In [52]:
recall

0.66

In [53]:
exported_rank[:20]

['98306:88871,50853,33630,90791|88871:0.6989353001117706,42169:0.6945495307445526,33257:0.6943211257457733,69738:0.6926285326480865,85708:0.686054915189743,52149:0.6835945248603821,85766:0.6826655864715576,64215:0.682452529668808,34754:0.6819010674953461,54812:0.6812411844730377,61075:0.6792391836643219,95954:0.6788847744464874,85098:0.6770849823951721,43048:0.6732473373413086,49047:0.6730431318283081,64391:0.6726735234260559,45548:0.6726596057415009,57003:0.6710462272167206,58811:0.6693163812160492,84938:0.6682612001895905,43589:0.6675132513046265,99507:0.6673316657543182,37509:0.666868656873703,75781:0.6666372716426849,48855:0.6663009226322174,113585:0.6660912334918976,55794:0.665748804807663,41139:0.6651560068130493,29075:0.6648722290992737',
 '32771:32490,33548,32560,33879,32699|33506:0.7244063317775726,9917:0.7180111110210419,39209:0.7102480828762054,12918:0.7066433429718018,17409:0.7064438462257385,12726:0.7056494355201721,4516:0.7055988013744354,71665:0.7042095363140106,86473:0.

### Retrieval evaluation

In [54]:
print("Total of queries:", len(retrieval.test))

Total of queries: 8265


#### Getting the model trained

In [55]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepTL_1000_feature_1000epochs_64batch(openoffice)'

In [56]:
encoded_anchor.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 738)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [57]:
len(exported_rank)

11757

In [58]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepTL_1000.txt'

In [59]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [60]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.54,
 '2 - recall_at_10': 0.59,
 '3 - recall_at_15': 0.62,
 '4 - recall_at_20': 0.65,
 '5 - recall_at_25': 0.66}

#### Some ideas to visualizate

- https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9